<h2>Runs the ti_melt model for a set of drainageIDs and years</h2>


In [ ]:
%pylab notebook
import matplotlib.pyplot as plt
from charistools.hypsometry import Hypsometry
from charistools.meltModels import TriSurfTempIndexMelt
from charistools.meltModels import ImshowTriSurfMelt
from charistools.meltModels import PlotTriSurfInput
from charistools.meltModels import PlotTriSurfMelt
from charistools.modelEnv import ModelEnv
from charistools.timeSeries import TimeSeries
import pandas as pd
import re
import os

In [ ]:
configFile = '/Users/brodzik/projects/CHARIS/charistools_test_data/UIB_GDBD_modelEnv_config.ini'
myEnv = ModelEnv(tileConfigFile=configFile)

In [ ]:
names = ['Astore', 'DrasNala', 'Gilgit', 'Hunza', 'Kharmong', 'Shigar', 'Shyok', 'Tarbela', 'Zanskar']
#names = ['DrasNala', 'Gilgit', 'Hunza', 'Kharmong', 'Shyok', 'Tarbela', 'Zanskar']
#names = ['Astore']
drainageIDs = ["IN_%s_GDBD" % name for name in names]
years = np.arange(14) + 2001
#years = np.arange(1) + 2001               
ablation_method = 'grsize_scag'
threshold = 205

In [ ]:
drainageIDs, years

In [ ]:
def run_model(drainageID, year=2001, ablation_method='grsize_scag', threshold=205, 
              model_str="8.67_8.67_10.0_17.0", label='best_model'):
    input = myEnv.model_inputs(drainageID=drainageID,
                               year=year,
                               ablation_method=ablation_method,
                               threshold=threshold)

    (min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf) = model_str.split("_")
    min_snow_ddf = float(min_snow_ddf)
    max_snow_ddf = float(max_snow_ddf)
    min_ice_ddf = float(min_ice_ddf)
    max_ice_ddf = float(max_ice_ddf)
    
    (SOLmelt, SOImelt, EGImelt) = TriSurfTempIndexMelt(
        input['snow_on_land_by_elevation_filename'],
        input['snow_on_ice_by_elevation_filename'],
        input['exposed_glacier_ice_by_elevation_filename'],
        input['temperature_by_elevation_filename'],
        min_snow_ddf=min_snow_ddf,
        max_snow_ddf=max_snow_ddf,
        min_ice_ddf=min_ice_ddf,
        max_ice_ddf=max_ice_ddf)
    
    SOLmeltfile = input['snow_on_land_by_elevation_filename']
    SOImeltfile = input['snow_on_ice_by_elevation_filename']
    EGImeltfile = input['exposed_glacier_ice_by_elevation_filename']
    SOLmeltfile = SOLmeltfile.replace('area_by_elev.', 'melt_by_elev.' + label + '.')
    SOImeltfile = SOImeltfile.replace('area_by_elev.', 'melt_by_elev.' + label + '.')
    EGImeltfile = EGImeltfile.replace('area_by_elev.', 'melt_by_elev.' + label + '.')

    columns = [float(i) for i in SOLmelt.data.columns]
    SOLmelt.data.columns = columns
    SOLmelt.data = SOLmelt.data[sort(columns)]

    # Temporary fix for SOLmelt.data NaNs
    SOLmelt.data = SOLmelt.data.fillna(value=0.)

    columns = [float(i) for i in SOImelt.data.columns]
    SOImelt.data.columns = columns
    SOImelt.data = SOImelt.data[sort(columns)]

    columns = [float(i) for i in EGImelt.data.columns]
    EGImelt.data.columns = columns
    EGImelt.data = EGImelt.data[sort(columns)]

    SOLmelt.write(filename=SOLmeltfile, decimal_places=2)
    SOImelt.write(filename=SOImeltfile, decimal_places=2)
    EGImelt.write(filename=EGImeltfile, decimal_places=2)
    
    print("%s : %d : model=%s" % (drainageID, year, model_str))
    
    baseFilename = SOImeltfile
    p = re.compile(r'snow_on_ice.+')
    baseFilename = p.sub('', baseFilename)
    
    return(baseFilename, input, model_str, SOLmelt, SOImelt, EGImelt)

In [ ]:
def show_melt_hyps(drainageID, year, baseFilename, SOLmelt, SOImelt, EGImelt):
    year_str = str(year)
    fig, axes = plt.subplots(3,1)
    ImshowTriSurfMelt(axes, SOLmelt, SOImelt, EGImelt)
    for ax in axes:
        ax.set_title(drainageID + " (" + year_str + ") " + ax.get_title())
    fig.tight_layout()
    outfile = baseFilename + 'melt_hyps.png'
    fig.savefig(outfile)
    print("Wrote melt_hyps to %s" % outfile)
    plt.close('all')

In [ ]:
def show_melt_tseries(drainageID, year, baseFilename, input, model_str, SOLmelt, SOImelt, EGImelt):
    year_str = str(year)
    (min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf) = model_str.split("_")
    min_snow_ddf = float(min_snow_ddf)
    max_snow_ddf = float(max_snow_ddf)
    min_ice_ddf = float(min_ice_ddf)
    max_ice_ddf = float(max_ice_ddf)
    
    melt_by_doy = (SOLmelt.data_by_doy() +
                   SOImelt.data_by_doy() +
                   EGImelt.data_by_doy())
    total_melt = melt_by_doy.sum()
    print("total melt = %.2f" % total_melt)
    melt_by_month = melt_by_doy.groupby([pd.TimeGrouper('M')]).sum().to_frame(name='melt')

    yyyymm_index = pd.to_datetime(melt_by_month.index.map(lambda x: x.strftime('%Y-%m-15')))
    df = pd.DataFrame(data=melt_by_month.values, index=yyyymm_index, columns=['melt'])
    df['SOLmelt'] = SOLmelt.data_by_doy().groupby([pd.TimeGrouper('M')]).sum().values
    df['SOImelt'] = SOImelt.data_by_doy().groupby([pd.TimeGrouper('M')]).sum().values
    df['EGImelt'] = EGImelt.data_by_doy().groupby([pd.TimeGrouper('M')]).sum().values
    #print(df)
    
    
    fig, ax = plt.subplots(3,1,figsize=(9,9))
    left_ax, right_ax = PlotTriSurfInput(ax[0], 
                             input['snow_on_land_by_elevation_hyps'],
                             input['snow_on_ice_by_elevation_hyps'],
                             input['exposed_glacier_ice_by_elevation_hyps'],
                             input['temperature_by_elevation_hyps'],
                             title="Inputs for %s (%d)" % (drainageID, year))
    
    h, l = left_ax.get_legend_handles_labels()                                       
    h1, l1 = right_ax.get_legend_handles_labels()                                    
    left_ax.legend(h+h1, l+l1, framealpha=0.5) 
    #left_ax.legend(framealpha=0.5)
    ax[1] = PlotTriSurfMelt(ax[1], SOLmelt, SOImelt, EGImelt, 
                            title="Modelled melt for %s (%d)" % (drainageID, year))
    ax[1].legend(framealpha=0.5)
    ax[1].text(ax[1].get_xlim()[0] + (0.03 * (ax[1].get_xlim()[1] - ax[1].get_xlim()[0])), 
               ax[1].get_ylim()[1] * 0.7,
               'snow DDF = %.2f - %.2f $mm/C/day$\nice DDF = %.2f - %.2f $mm/C/day$' % 
               (min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf),
               style='italic',
               bbox={'facecolor':'gray', 'alpha':0.1, 'pad':10})
    
    # Get the line colors used by PlotTriSurfMelt
    lines = ax[1].get_lines()
    SOIcolor = lines[1].get_color()
    EGIcolor = lines[2].get_color()
    SOLcolor = lines[3].get_color()
    
    df[['EGImelt', 'SOImelt', 'SOLmelt']].plot(ax=ax[2], 
                                               stacked=True, kind="bar", 
                                               position=0.25,
                                               edgecolor="white",
                                               color=[EGIcolor, SOIcolor, SOLcolor])
    ax[2].set_title("Monthly Melt")
    ax[2].set_ylabel('Volume (' + r'$km^3$' + ')') 
    ax[2].xaxis.set_major_formatter(plt.NullFormatter())
    for container in ax[2].containers:
        plt.setp(container, width=0.25)
    ax[2].set_ylim([0,1.1 * np.amax(df["melt"])])
    ax[2].text(ax[2].get_xlim()[0] + (0.03 * (ax[2].get_xlim()[1] - ax[2].get_xlim()[0])), 
               ax[2].get_ylim()[1] * 0.7,
               'melt = %.2f $km^3$' % 
               (df["melt"].sum()),
               style='italic',
               bbox={'facecolor':'gray', 'alpha':0.1, 'pad':10})
    handles, labels = ax[2].get_legend_handles_labels()
    ax[2].legend(reversed(handles), reversed(labels))

    fig.tight_layout()
    outfile = baseFilename + 'melt_tseries.png'
    fig.savefig(outfile)
    print("Wrote melt_tseries to %s" % outfile)
    plt.close('all')

In [ ]:
for drainageID in drainageIDs:
    for year in years:
        (baseFilename, input, model_str, SOLmelt, SOImelt, EGImelt) = run_model(
            drainageID=drainageID, year=year, ablation_method=ablation_method, threshold=threshold)
        show_melt_hyps(drainageID, year, baseFilename, SOLmelt, SOImelt, EGImelt)
        show_melt_tseries(drainageID, year, baseFilename, input, model_str, SOLmelt, SOImelt, EGImelt)

In [ ]:
12./61.